In [55]:
#imports and user defined errors 
import re

class GateNotDefined(Exception):
    '''
    Error to be raised when the gate is not found in the the basic gates 
    '''
    def __init__(self, gate):
        self.message = f"{gate} not defined"
        super().__init__(self.message)

class GateReprError(Exception):
    '''
    Error to be raised for illegal gate declaration format 
    '''
    def __init__(self, gate_repr):
        self.message = f"{gate_repr} not valid"
        super().__init__(self.message)

#custom errors to be used
class FloatingOutput(Exception):
    '''
    Error to be raised when an output of a gate is floating
    '''
    def __init__(self, node_name):
        self.message = f"{node_name} is floating, no driver for output node is defined"
        super().__init__(self.message)

class CirNotLevelized(Exception):
    '''
    Error to be raised if a function that requires levelized circuit is called
    before the circuit is levelized
    '''
    def __init__(self):
        self.message = f"Circuit is not levelized! Levelize circuit and try again"
        super().__init__(self.message)


In [56]:
#basic classes used for circuit representation
class node_type:
    input_node = 0
    output_node = 1
    internal_wire = 3

#class to represnt the value of each node
class node_value:
    zero = 0
    one = 1
    d = 2
    d_bar = 3
    undefined = -1

class gate_type:
    AND_gate = 0
    OR_gate = 1
    NOT_gate = 2
    NAND_gate = 3
    NOR_gate = 4
    XOR_gate = 5
    XNOR_gate = 6

    @staticmethod
    def AND(input_list):
        output = node_value.one

        for n in input_list:
            if n.value == node_value.zero:
                return node_value.zero
            elif n.value == node_value.undefined:
                return node_value.undefined
            elif n.value == node_value.one:
                continue
            elif n.value == node_value.d:
                if output == node_value.d_bar:
                    return node_value.zero
                output = node_value.d
            elif n.value == node_value.d_bar:
                if output == node_value.d:
                    return node_value.zero
                output = node_value.d_bar

        return output
    
    @staticmethod
    def OR(input_list):
        output = node_value.zero

        for n in input_list:
            if n.value == node_value.one:
                return node_value.one
            elif n.value == node_value.undefined:
                return node_value.undefined
            elif n.value == node_value.zero:
                continue
            elif n.value == node_value.d:
                if output == node_value.d_bar:
                    return node_value.one
                output = node_value.d
            elif n.value == node_value.d_bar:
                if output == node_value.d:
                    return node_value.one
                output = node_value.d_bar

        return output
    
    @staticmethod
    def NOT(input_list):
        output_dict = {node_value.zero: node_value.one,
                       node_value.one: node_value.zero,
                       node_value.d: node_value.d_bar,
                       node_value.d_bar: node_value.d,
                       node_value.undefined: node_value.undefined}
        
        return output_dict[input_list[0].value]
    
    @staticmethod
    def NAND(input_list):
        output = node_value.one

        for n in input_list:
            if n.value == node_value.zero:
                return node_value.one
            elif n.value == node_value.undefined:
                return node_value.undefined
            elif n.value == node_value.one:
                continue
            elif n.value == node_value.d:
                if output == node_value.d_bar:
                    return node_value.one
                output = node_value.d_bar
            elif n.value == node_value.d_bar:
                if output == node_value.d:
                    return node_value.one
                output = node_value.d

        return output
    
    @staticmethod
    def NOR(input_list):
        output = node_value.zero

        for n in input_list:
            if n.value == node_value.one:
                return node_value.zero
            elif n.value == node_value.undefined:
                return node_value.undefined
            elif n.value == node_value.zero:
                continue
            elif n.value == node_value.d:
                if output == node_value.d_bar:
                    return node_value.zero
                output = node_value.d_bar
            elif n.value == node_value.d_bar:
                if output == node_value.d:
                    return node_value.zero
                output = node_value.d

        return output
    
    @staticmethod
    def XOR(input_list):
        inverted_output = {node_value.zero: node_value.one,
                          node_value.one: node_value.zero,
                          node_value.d: node_value.d_bar,
                          node_value.d_bar: node_value.d,
                          node_value.undefined: node_value.undefined}
        
        output = node_value.zero

        for n in input_list:
            if n.value == node_value.undefined:
                return node_value.undefined
            elif n.value == node_value.zero:
                continue
            elif n.value == node_value.one:
                output = inverted_output[output]
            elif n.value == node_value.d:
                if output == node_value.zero:
                    output = node_value.d
                elif output == node_value.one:
                    output = node_value.d_bar
                elif output == node_value.d:
                    output = node_value.zero
                elif output == node_value.d_bar:
                    output = node_value.one
    
        return output
    
    @staticmethod
    def XNOR(input_list):
        inverted_output = {node_value.zero: node_value.one,
                          node_value.one: node_value.zero,
                          node_value.d: node_value.d_bar,
                          node_value.d_bar: node_value.d,
                          node_value.undefined: node_value.undefined}
        
        output = node_value.zero

        for n in input_list:
            if n.value == node_value.undefined:
                return node_value.undefined
            elif n.value == node_value.zero:
                output = inverted_output[output]
            elif n.value == node_value.one:
                continue
            elif n.value == node_value.d:
                if output == node_value.zero:
                    output = node_value.d_bar
                elif output == node_value.one:
                    output = node_value.d
                elif output == node_value.d:
                    output = node_value.one
                elif output == node_value.d_bar:
                    output = node_value.zero
            elif n.value == node_value.d_bar:
                if output == node_value.zero:
                    output = node_value.d
                elif output == node_value.one:
                    output = node_value.d_bar
                elif output == node_value.d:
                    output = node_value.zero
                elif output == node_value.d_bar:
                    output = node_value.one

        return output
                
class fault_types:
    sa0 = 0
    sa1 = 1

In [57]:
#class to represent a fault
class fault:
    '''
    This class represents a fault.

    Attributes:
    fault_type: Type of fault, either stuck at 0 or stuck at 1
    fault_node: Node to which the fault is associated with
    fault_output: Output linked with the node where the fault is associated
    '''

    def __init__(self, fault_type, fault_node, fault_output = None):
        self.fault_type = fault_type
        self.fault_node = fault_node
        self.fault_output = fault_output

    def __repr__(self):
        '''
        Returns the string representation of a fault
        '''
        if self.fault_output == None:
            if self.fault_type == fault_types.sa0:
                return f"{self.fault_node}-0"
            else:
                return f"{self.fault_node}-1"
        else:
            if self.fault_type == fault_types.sa0:
                return f"{self.fault_output}-{self.fault_node}-0"
            else:
                return f"{self.fault_output}-{self.fault_node}-1"

In [58]:
#class to represent a gate
class gate:
    '''
    This class is used to represent a gate

    Attributes:
    num_inputs: Number of inputs(int)
    type: Type of the gate(gate_type)
    input_nodes: Nodes that are fed in
    Output node: Output node of the gate
    '''

    def __init__(self, string_representation):
        '''
        This function creates a gate

        param[in] string_representation String representation of the gate
                                        The string after = in the circuit bench file
        '''
        self.input_nodes = []
        self.output_node = None
        self.type = None
        self.num_inputs = 0
        self.gate_string = ""

        self.__update_gate_type(string_representation)

    def __update_gate_type(self, string_representation: str):
        '''
        This function updates the gate type
        '''
        gate_dict = {"AND" : gate_type.AND_gate,
                     "OR"  : gate_type.OR_gate,
                     "NOT" : gate_type.NOT_gate,
                     "NOR" : gate_type.NOR_gate,
                     "NAND": gate_type.NAND_gate,
                     "XOR" : gate_type.XOR_gate,
                     "XNOR": gate_type.XNOR_gate}

        self.gate_string = re.match(r'^[^()]+', string_representation)

        if self.gate_string:
            self.gate_string = self.gate_string.group()
        else:
            raise GateReprError(string_representation)
        

        if self.gate_string not in gate_dict:
            raise GateNotDefined(self.gate_string)
        else:
            self.type = gate_dict[self.gate_string]
            self.input_nodes = re.findall(r'\((.*?)\)', string_representation)[0]
            self.input_nodes = self.input_nodes.split(",")
            for i in range(len(self.input_nodes)):
                self.input_nodes[i] = self.input_nodes[i].lstrip()
            self.num_inputs = len(self.input_nodes)

    def __repr__(self):
        '''
        Return a string representing the gate information
        '''
        str_repr = f"{self.num_inputs}-input {self.gate_string} gate | Input nodes: "
        for node in self.input_nodes:
            str_repr = str_repr + f"{node},"
        str_repr = str_repr[:-1] + " | "
        str_repr = f"{str_repr}Output node: {self.output_node}"
        return str_repr

In [59]:
#class to represent a node
class node:
    '''
    This class represents a node in the circuit

    Attributes:
    name: Name of the node(string)
    type: Type of the node(node_type)
    value: Value of the node(node_value)
    gate_type: Type of gate(String)
    nodes_fed_in: List of nodes fed in(node)
    fault_list: List of faults associated with this node
    '''

    def __init__(self, circuit_bench_line: str):
        '''
        Initialize the object. 

        param[in] circuit_bench_line: A line in the circuit bench file
        '''
        self.name = None
        self.type = None
        self.value = node_value.undefined
        self.gate = None
        self.level = None
        self.fault_list = {fault_types.sa0: [], fault_types.sa1: []}

        #check if it is an input node
        if ("INPUT" in circuit_bench_line.upper()):
            self.type = node_type.input_node
            self.name = re.findall(r'\((.*?)\)', circuit_bench_line)[0]
            self.level = 0
        elif ("OUTPUT" in circuit_bench_line.upper()):
            self.type = node_type.output_node
            self.name = re.findall(r'\((.*?)\)', circuit_bench_line)[0]
        else:
            #its an internal wire
            self.type = node_type.internal_wire
            self.name = circuit_bench_line.split("=")[0] #split at "=" symbol and the first element in the list
                                                         #is the node name
            self.name = self.name.rstrip() #remove any trailing white spaces
            self.__update_gate(circuit_bench_line)

    def __update_gate(self, circuit_bench_line):
        '''
        Updates the gate information of a node
        '''
        gate_string_representation = circuit_bench_line.split("=")[1]
        gate_string_representation = gate_string_representation.lstrip()
        self.gate = gate(gate_string_representation)
        self.gate.output_node = self

    def update(self, circuit_bench_line):
        '''
        This function updates the nodes properties based on the new circuit bench file
        '''
        if "OUTPUT" in circuit_bench_line.upper():
            #node is an output node
            self.type = node_type.output_node
        else:
            #an assignment done to the node, update the gate type
            self.__update_gate(circuit_bench_line)

    def find_level(self):
        if self.level != None:
            #level already found
            return
        
        prev_level = 0

        for input_node in self.gate.input_nodes:
            if input_node.level == None:
                input_node.find_level()

            if input_node.level > prev_level:
                prev_level = input_node.level   

        self.level = prev_level + 1

    def create_fault_list(self, output_node = None):
        if output_node == None:
            if len(self.fault_list[fault_types.sa0]) == 0:
                self.fault_list[fault_types.sa0].append(fault(fault_types.sa0, self)) #creates a-0 fault
                if self.type == node_type.output_node:
                    self.fault_list[fault_types.sa0].append(fault(fault_types.sa0, self, "out"))
            if len(self.fault_list[fault_types.sa1]) == 0:
                self.fault_list[fault_types.sa1].append(fault(fault_types.sa1, self)) #creates a-1 fault
                if self.type == node_type.output_node:
                    self.fault_list[fault_types.sa1].append(fault(fault_types.sa1, self, "out"))
        else:
            self.fault_list[fault_types.sa0].append(fault(fault_types.sa0, self, output_node))
            self.fault_list[fault_types.sa1].append(fault(fault_types.sa1, self, output_node))
            return
        
        if self.gate != None:
            for input_node in self.gate.input_nodes:
                input_node.create_fault_list(self) #creates g-a-0 and g-a-1
        


    def __repr__(self):
        return self.name

In [60]:
#class to represent a circuit
class circuit:
    '''
    This class represents a circuit described in the circuit bench file

    Attributes:
    nodes: List of all the nodes(nodes)
    input_list: Index of input nodes in nodes list(int)
    output_list: Index of output nodes in nodes list(int)    
    '''
    
    def __init__(self, circuit_bench_file):
        '''
        Initializes the class

        param[in] circuit_bench_file: Path to the circuit bench file
        '''
        self.nodes = []
        self.input_list = []
        self.output_list = []
        self.node_index = {} #index of each node in the nodes list, key values for this dictionary are the node names
        self.levelized_nodes = None
        
        lines = self.__parse_circuit_bench_file(circuit_bench_file)
        #create nodes
        self.__create_nodes(lines)
        #check if all outputs are defined
        self.__check_output_definition()

    def __parse_circuit_bench_file(self, circuit_bench_file):
        '''
        This function parses the circuit bench file and sends out a list
        '''
        lines = []

        with open(circuit_bench_file, "r") as f:
            for line in f:
                line = line.replace("\n", "")
                if len(line) == 0:
                    continue
                lines.append(line)

        return lines
    
    def __create_nodes(self, lines):
        '''
        This function creates nodes from the lines read from the circuit bench file
        '''
        for line in lines:
            #check if node is already created - Possible when output is declared after using a wire
            node_name = re.findall(r'\((.*?)\)', line)[0]

            if "=" in line:
                node_name = line.split("=")[0]
                node_name = node_name.rstrip()

            if node_name in self.node_index:
                #update the node properties
                self.nodes[self.node_index[node_name]].update(line)
            else:
                #new node in the circuit
                #create a node and add it to the node list
                self.nodes.append(node(line))
                if self.nodes[-1].type == node_type.input_node:
                    self.input_list.append(self.nodes[-1].name)
                elif self.nodes[-1].type == node_type.output_node:
                    self.output_list.append(self.nodes[-1].name)
                self.node_index[self.nodes[-1].name] = len(self.nodes) - 1

        #all the nodes are defined, now update the input nodes in the gates list to nodes
        #before this point, the input nodes in gate contains the node name alone and not the
        #node object
        for n in self.nodes:
            if n.gate != None:
                nodes_fed_in = n.gate.input_nodes
                for i in range(len(nodes_fed_in)):
                    nodes_fed_in[i] = self.nodes[self.node_index[nodes_fed_in[i]]]
                n.gate.input_nodes = nodes_fed_in

    
    def __check_output_definition(self):
        '''
        This function checks if all outputs are defined
        '''
        for node in self.nodes:
            if node.type == node_type.output_node:
                if node.gate == None:
                    raise FloatingOutput(node.name)
                
    def __repr__(self):
        '''
        This function returns the string representation of a circuit
        '''
        str_repr = "-------------------------------------------\n"
        str_repr = f"{str_repr}--------------Input Nodes------------------\n{self.input_list}\n"
        str_repr = f"{str_repr}-------------Output Nodes------------------\n{self.output_list}\n"
        str_repr = f"{str_repr}---------------Gate list-------------------\n"

        for node in self.nodes:
            if node.type != node_type.input_node:
                str_repr = f"{str_repr}{node.gate}\n"
        
        if self.levelized_nodes != None:
            str_repr = f"{str_repr}-------------Levelized circuit-------------\n"
            for level in self.levelized_nodes:
                str_repr = f"{str_repr}Level {level}: {self.levelized_nodes[level]}\n"

        return str_repr
    
    def levelize_circuit(self):
        '''
        This function levelizes the circuit and updates levelized_nodes
        '''
        if self.levelized_nodes != None:
            #circuit already levelized
            return
        
        self.levelized_nodes = {}
        
        for node in self.nodes:
            if node.level == None:
                node.find_level()

            if node.level in self.levelized_nodes:
                self.levelized_nodes[node.level].append(node.name)
            else:
                self.levelized_nodes[node.level] = [node.name]

        self.levelized_nodes = dict(sorted(self.levelized_nodes.items()))

    def create_fault_list(self):
        '''
        This function creates the fault list for all the nodes in the circuit
        '''
        if self.levelized_nodes == None:
            raise CirNotLevelized()
        for level in self.levelized_nodes:
            for n in self.levelized_nodes[level]:
                self.nodes[self.node_index[n]].create_fault_list()

In [61]:
circuit_under_test = circuit("p1.txt")
circuit_under_test.levelize_circuit()
print(circuit_under_test)

-------------------------------------------
--------------Input Nodes------------------
['a', 'b', 'c']
-------------Output Nodes------------------
['w', 'x', 'y', 'z']
---------------Gate list-------------------
3-input AND gate | Input nodes: a,b,c | Output node: w
3-input OR gate | Input nodes: d,e,f | Output node: x
2-input OR gate | Input nodes: g,h | Output node: y
1-input NOT gate | Input nodes: c | Output node: z
1-input NOT gate | Input nodes: a | Output node: a'
1-input NOT gate | Input nodes: b | Output node: b'
1-input NOT gate | Input nodes: c | Output node: c'
2-input AND gate | Input nodes: a,b' | Output node: d
3-input AND gate | Input nodes: a',b,c | Output node: e
3-input AND gate | Input nodes: a,b,c' | Output node: f
2-input AND gate | Input nodes: b',c | Output node: g
2-input AND gate | Input nodes: b,c' | Output node: h
-------------Levelized circuit-------------
Level 0: ['a', 'b', 'c']
Level 1: ['w', 'z', "a'", "b'", "c'"]
Level 2: ['d', 'e', 'f', 'g', 'h']
Lev

In [62]:
circuit_under_test.create_fault_list()

In [63]:
total_faults = 0
for level in circuit_under_test.levelized_nodes:
    for n in circuit_under_test.levelized_nodes[level]:
        print(circuit_under_test.nodes[circuit_under_test.node_index[n]].fault_list)
        total_faults += len(circuit_under_test.nodes[circuit_under_test.node_index[n]].fault_list[0]) * 2

print(total_faults)

{0: [a-0, w-a-0, a'-a-0, d-a-0, f-a-0], 1: [a-1, w-a-1, a'-a-1, d-a-1, f-a-1]}
{0: [b-0, w-b-0, b'-b-0, e-b-0, f-b-0, h-b-0], 1: [b-1, w-b-1, b'-b-1, e-b-1, f-b-1, h-b-1]}
{0: [c-0, w-c-0, z-c-0, c'-c-0, e-c-0, g-c-0], 1: [c-1, w-c-1, z-c-1, c'-c-1, e-c-1, g-c-1]}
{0: [w-0, out-w-0], 1: [w-1, out-w-1]}
{0: [z-0, out-z-0], 1: [z-1, out-z-1]}
{0: [a'-0, e-a'-0], 1: [a'-1, e-a'-1]}
{0: [b'-0, d-b'-0, g-b'-0], 1: [b'-1, d-b'-1, g-b'-1]}
{0: [c'-0, f-c'-0, h-c'-0], 1: [c'-1, f-c'-1, h-c'-1]}
{0: [d-0, x-d-0], 1: [d-1, x-d-1]}
{0: [e-0, x-e-0], 1: [e-1, x-e-1]}
{0: [f-0, x-f-0], 1: [f-1, x-f-1]}
{0: [g-0, y-g-0], 1: [g-1, y-g-1]}
{0: [h-0, y-h-0], 1: [h-1, y-h-1]}
{0: [x-0, out-x-0], 1: [x-1, out-x-1]}
{0: [y-0, out-y-0], 1: [y-1, out-y-1]}
86
